# Experiments

In [7]:
import os
import pandas as pd
import time
import torch
from configs.models import SCOOP_cfg, NeuralPrior_cfg
from data.gpu_utils import get_free_gpu_indices
from pipeline.run_utils import run_experiment


# load cfg list from csv later? or just permute variables
cfg_list = [#{**SCOOP_cfg, 'affiliation' : 'baseline', 'smooth_weight' : 1},
            # {**NeuralPrior_cfg, 'affiliation' : 'baseline'},

            # {**SCOOP_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'lr' : 0.08},
            # {**SCOOP_cfg, 'affiliation' : 'ours', 'free_weight' : 1},
            # {**SCOOP_cfg, 'affiliation' : 'ours', 'forward_weight' : 1, 'pc2_smooth' : 1},
            # {**SCOOP_cfg, 'affiliation' : 'ours', 'forward_weight' : 10, 'pc2_smooth' : 1},
            # ctrl+d to clone

            # {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'K' : 32, 'smooth_weight' : 10., 'forward_weight' : 0, 'pc2_smooth' : 1},
            # {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'K' : 32, 'smooth_weight' : 3., 'forward_weight' : 1, 'pc2_smooth' : 1},
            # {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'K' : 32, 'smooth_weight' : 1., 'forward_weight' : 1, 'pc2_smooth' : 1},
            # {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'K' : 32, 'smooth_weight' : 1., 'forward_weight' : 1, 'pc2_smooth' : False},
            {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 5, 'K' : 4, 'smooth_weight' : 1., 'forward_weight' : 1, 'free_weight' : 1, 'pc2_smooth' : 1, 'VA' : 1},


            # {**SCOOP_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 8, 'forward_weight' : 1, 'pc2_smooth' : 1, 'lr' : 0.08},
            ]

import subprocess
DETACH = 0
# tohle cele do subprocessu ve funkci potom #

for cfg in cfg_list:

    while len(get_free_gpu_indices()) == 0:
        print('waiting for gpu')
        time.sleep(20)


    # for dataset in ['kitti_o', 'kitti_t', 'argoverse', 'nuscenes', 'waymo']:
    for dataset in ['kitti_t']:

        cfg['dataset'] = dataset
        cfg['iters'] = 10
        # cfg['lr'] = 0.008
        cfg['dev'] = 1
        print(dataset)
        run_experiment(cfg, DETACH=DETACH)

    # time.sleep(5)

    # break

kitti_t


0it [00:00, ?it/s]
Traceback (most recent call last):
  File "/mnt/home.dokt/vacekpa2/pcflow/pipeline/sceneflow.py", line 446, in <module>
    Experiment.optimize_sceneflow()
  File "/mnt/home.dokt/vacekpa2/pcflow/pipeline/sceneflow.py", line 290, in optimize_sceneflow
    pred_dict = solver(pc1, pc2, gt_flow, self.model, self.Loss_Function, self.args)
  File "/mnt/home.dokt/vacekpa2/pcflow/pipeline/sceneflow.py", line 135, in solver
    Loss_Function.update(pc1, pc2)
  File "/home.dokt/vacekpa2/pcflow/loss/flow.py", line 521, in update
    self.VAChamfer_loss = VAChamferLoss(pc2, **self.kwargs)
  File "/home.dokt/vacekpa2/pcflow/loss/flow.py", line 400, in __init__
    self.Visibility = VisibilityScene(dataset=self.kwargs['dataset'], pc_scene=pc2)
  File "/home.dokt/vacekpa2/pcflow/data/range_image.py", line 315, in __init__
    self.depth_image = self.calculate_depth_image()
  File "/home.dokt/vacekpa2/pcflow/data/range_image.py", line 389, in calculate_depth_image
    self.depth_ima

In [ ]:
# # Visualize
from vis.deprecated_vis import *
from data.PATHS import TMP_VIS_PATH
import numpy as np
import glob


last_exp = sorted(glob.glob(f'{os.path.expanduser("~")}/experiments/SCOOP/*'))[-1]
data = np.load(f'{last_exp}/inference/000000.npz', allow_pickle=True)
pc1 = data['pc1']
pc2 = data['pc2']
est_flow1 = data['est_flow']

# plot epe and loss
epe_all = data['epe_all']
loss_all = data['loss_all']

import matplotlib.pyplot as plt
fig, axes = plt.subplots(ncols=2)

axes[0].plot(epe_all, 'r--')
axes[1].plot(loss_all, 'b--')
axes[0].set_title('EPE per iter')
axes[1].set_title('Loss per iter')


axes[0].set_xlabel('iter')
axes[1].set_xlabel('iter')
axes[0].set_ylabel('EPE')
axes[1].set_ylabel('loss')
axes[0].legend(['EPE'])
axes[1].legend(['Loss'])
axes[0].grid(True)
axes[1].grid(True)

plt.savefig(f'{TMP_VIS_PATH}/epe_loss.png')
plt.close()
# visualize_flow3d(pc1, pc2, est_flow1)



# notes
- SCOOP wont generalize to other real world datasets (Cross-dataset evaluation) - performance on argoverse is bad, retraining?
    - "It extracts discriminative features, which transfer well across the FT3Do and KITTIo datasets, and enables to compute the correspondence-based flow between the point clouds"
    - We dont have advantage in that, but this kinda justifies the results on argoverse without fine-tunning?
    - KittiSF is bijection, that is why it works very good in SCOOP
- https://arxiv.org/pdf/2305.02528.pdf cite baseline
- Pouceni z reimplementace
    - Asi to fakt chce delat krok po kroku uz odzacatku? Tim mysleno testovat vsechno krok po kroku kdyz se prevezmou funkce
    - Izolovat (rozumet tem krokum, jinak je proste nedelat, opravdu nedelat)
    - Skvely je mit ten framework udelany podle toho, co opravdu clovek delat
    - Nejlepsi je mit vsechno v jednom editoru viz. poustet pre python subprocessy, jinak prepinani zpomaluje a nici to framework workflow


In [ ]:
# Compare experiments, # Udelat cross experiment vizualizaci
import glob
import pandas as pd
import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH

exp_name = "NeuralPrior"
exp_dir = f'{os.path.expanduser("~")}/experiments/{exp_name}'
runs = sorted(glob.glob(f'{exp_dir}/*'))

# plot?

metric_all = []
fig, axes = plt.subplots(1, 2)#, figsize=(10, 5))

for run in runs:

    if os.path.exists(f'{run}/metric.csv') == False:
        continue

    metric = pd.read_csv(f'{run}/metric.csv', index_col=0, header=None).transpose()
    args = pd.read_csv(f'{run}/args.csv', index_col=0, header=None).transpose()


    # Skip tryouts
    dev = int(args['dev'].values[0])

    if dev == 1:
        continue

    aff = args['affiliation'].values[0]
    epe = metric['EPE3D'].values[0]

    avg_solver_time = metric['avg_solver_time'].values[0]

    if aff == 'baseline':
        axes[0].plot(avg_solver_time, epe, 'b*', markersize=10)

    if aff == 'ours':
        axes[0].plot(avg_solver_time, epe, 'g*', markersize=10)


    print_metric = metric
    print_metric['aff'] = aff
    print_metric['free'] = args.free_weight
    print_metric['forward'] = args.forward_weight
    print_metric['pc2_smooth'] = args.pc2_smooth
    print_metric['free'] = args.free_weight

    metric_all.append(print_metric)

df = pd.concat(metric_all, ignore_index=True)



axes[0].set_xlabel('Average Solver Time Per Frame [s]')
axes[0].set_ylabel('EPE [m]')

axes[0].set_title('Performance on KITTI-SF')
axes[0].legend(['baseline', 'ours'])
axes[0].grid(True)


#create pandas DataFrame

#create table
# table = plt.table(cellText=df.values, colLabels=df.columns, loc='center')
plt.savefig(f'{TMP_VIS_PATH}/kittisf.png')
plt.close()

df


In [ ]:
res_path = sorted(glob.glob(os.path.expanduser("~") + '/experiments/SCOOP/2023-07-13-12-55-18-642/inference/*'))
# kitti_t = [file for file in sorted(glob.glob(os.path.expanduser("~") + '/pcflow/models/SCOOP/pretrained_models/kitti_v_100_examples/pc_res/*.npz'))]
# kitti_t = [int(os.path.basename(file).split('_')[0]) for file in sorted(glob.glob(os.path.expanduser("~") + '/pcflow/models/SCOOP/pretrained_models/kitti_v_100_examples/pc_res/*.npz'))]

kitti_t_files = sorted(glob.glob(os.path.expanduser("~") + '/pcflow/models/SCOOP/pretrained_models/kitti_v_100_examples/pc_res/*.npz'))

epe_list = []
# visualize one frame
for idx in range(len(kitti_t_files)):

    # My
    # data = np.load(f'{res_path[idx]}', allow_pickle=True)
    # epe = data['EPE3D']

    # Former
    data = np.load(f'{kitti_t_files[idx]}', allow_pickle=True)
    pred_flow = data['est_flow_for_pc1']
    gt_flow = data['gt_flow_for_pc1']
    epe = np.linalg.norm(pred_flow - gt_flow, axis=1).mean()

    # if idx == 43:
    # print(data['pc1'].shape, data['pc2'].shape)


    epe_list.append(epe)

    # print(os.path.basename(res_path[idx]), epe)

    # to get pitch field of view
    # pc2 = data['pc2']
    # pitch = np.arcsin(pc2[:, 1] / np.linalg.norm(pc2, axis=1))
    # print(pitch.max() * 180 / np.pi, pitch.min() * 180 / np.pi)

plt.plot(epe_list, 'b*')
plt.title(f'EPE on KITTI-SF --- {np.array(epe_list).mean():.5f}m')
plt.xlabel('Frame [-]')
plt.ylabel('EPE [m]')

plt.savefig(f'{TMP_VIS_PATH}/epe.png')
plt.close()




In [ ]:
import itertools
import pandas as pd
### Make configs

permute_cfg = {'model': ['NeuralPrior', 'SCOOP'],
               'sm_normals_K': [0, 3, 5, 8, 12],
               'FSM':  [0, 1, 2, 4, 10],
               'pc2_smooth': [0, 1],
               'K': [0, 4, 6, 8, 32],
               'Free_weight': [0, 1, 2, 5],
               'VA_SM': [0, 1],
               'lr': [0.2, 0.08, 0.008, 0.001, 0.003],
               'dataset': ['kitti_t', 'kitti_o', 'argoverse', 'nuscenes', 'waymo'],
               'max_radius' : [2],
               'run': [0]}  # use 3 runs, each server one


# permute all possible combinations of variables above

index = permute_cfg.keys()
combinations = list(itertools.product(*permute_cfg.values()))

df = pd.DataFrame(combinations, columns=index)

valid_cfg_list = []

# leave out some configs
for i in range(len(combinations)):

    c = df.iloc[i]

    ##### Kittisf
    if c['dataset'].startswith('kitti'):

        if c['K'] != 32: continue

    ##### Lidar
    if not c['dataset'].startswith('kitti'):

        if c['K'] > 8: continue
        if c['sm_normals_K'] > 5: continue

    ##### Models
    if c['model'] == "SCOOP":

        if c['K'] != 32: continue
        if c['lr'] != 0.2: continue
        if c['dataset'] in ['argoverse', 'nuscenes', 'waymo']: continue

    if c['model'] == 'NeuralPrior':

        if c['dataset'] in ['kitti_o']: continue
        if c['lr'] != 0.001 and c['dataset'] == 'kitti_t': continue
        if c['lr'] != 0.003 and c['dataset'] in ['argoverse', 'waymo', 'nuscenes']: continue


    ##### General
    if c['FSM'] == 0 and c['pc2_smooth'] == 1: continue
    if c['K'] == 0 and (c['FSM'] > 0 or c['VA_SM'] > 0): continue


    # you can add default configs

    valid_cfg_list.append(c.to_dict())

print(len(valid_cfg_list))
nbr_gpu = 5
experiment_time = 10 / 60 # hours
gpu_time = len(valid_cfg_list) * experiment_time

print(f"Runs: {len(valid_cfg_list)} ---> GPU time: {gpu_time:.1f} hours ---> {gpu_time / nbr_gpu / 24:.1f} days")

final_df = pd.DataFrame(valid_cfg_list, columns=index)

In [ ]:

# Top down view example of NN connections
KNN = nn_ind[0]
start_arrows = pc2[0][KNN].reshape(-1,3)

# NN connections
residuals_KNN = (pc2[0].unsqueeze(1) - pc2[0][KNN]).reshape(-1,3)
residuals_VA_KNN = (pc2[0].unsqueeze(1) - pc2[0][VA_KNN]).reshape(-1,3)

import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH


fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].plot(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), 'bo', alpha=0.15, markersize=0.1)
ax[1].plot(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), 'bo', alpha=0.15, markersize=0.1)

ax[0].quiver(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), residuals_KNN[:,0].cpu(), residuals_KNN[:,1].cpu(), color='r', scale_units='xy')
ax[1].quiver(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), residuals_VA_KNN[:,0].cpu(), residuals_VA_KNN[:,1].cpu(), color='r', scale_units='xy')

ax[0].axis('equal')
ax[1].axis('equal')

plt.savefig(f'{TMP_VIS_PATH}/{dataset_type}_range.png')


In [26]:
# Function to reassign depth to point clouds - it is somewhere
# produce same outputs so you dont have to recalculate them in each experiment
from data.range_image import *
from data.params.kittisf import P_rect

from vis.deprecated_vis import *
import numpy as np
from data.PATHS import TMP_VIS_PATH


depth_image = np.load("/home.dokt/vacekpa2/data/kittisf/all_data_format/000000.npz")['depth1']


orig_pc2 = np.load("/home.dokt/vacekpa2/data/kittisf/all_data_format/000000.npz")['pc1']

depth_image = torch.from_numpy(depth_image)
orig_pc2 = torch.from_numpy(orig_pc2)
# print(orig_pc2.shape)
# recon_pc = pixel2xyz(depth_image, P_rect)
# recon_pc = recon_pc.reshape(-1,3)
recon_pc = orig_pc2
# new_depth_image, px, py = xyz2pixel(recon_pc[:], P_rect, depth_image.shape[0], depth_image.shape[1])

plt.close()
plt.clf()
dataset_type = 'kitti_t'

Visibility = VisibilityModule(dataset = dataset_type)


# new_depth_image, px, py, inside = Visibility.generate_range_coors(recon_pc[None, :, :])
# from data.range_image import xyz2pixel
# new_depth_image, px, py, mask = xyz2pixel(recon_pc[:, :], P_rect, depth_image.shape[0], depth_image.shape[1])

recon_pc = recon_pc# + torch.rand(recon_pc.shape)
pc = recon_pc
height, width = depth_image.shape[0], depth_image.shape[1]
x = - pc[:,0]
y = - pc[:,1]
z = pc[:,2]

depth = z   # in this case
# depth = pc.norm(dim=-1)   # in this case
focal_length_pixel = P_rect[0, 0]
# height, width = depth_image.shape[:2]

const_x = P_rect[0, 2] * depth + P_rect[0, 3]
const_y = P_rect[1, 2] * depth + P_rect[1, 3]

# const_x = P_rect[0, 2]
# const_y = P_rect[1, 2]

px = (x * focal_length_pixel + const_x) / (depth + P_rect[2, 3])
py = (y * focal_length_pixel + const_y) / (depth + P_rect[2, 3])

# px = (x / z) * focal_length_pixel + const_x
# py = (y / z) * focal_length_pixel + const_y

# drop outside image points
mask = (px > 0) & (py > 0) & (px < width) & (py < height)

new_depth_image = - torch.ones((height, width), dtype=torch.float, device=pc.device)
new_depth_image[py[mask].to(int), px[mask].to(int)] = depth[mask]



# visualize_points3D(recon_pc, mask)
fig, ax = plt.subplots(3,1)
ax[0].imshow(depth_image), ax[0].set_title('Orig depth')
ax[1].imshow(new_depth_image), ax[1].set_title('Recon depth')
ax[2].imshow(depth_image - new_depth_image, interpolation='none'), ax[2].set_title('Diff')

plt.savefig(f'{TMP_VIS_PATH}/{dataset_type}_depth.png')

In [4]:
from data.range_image import VisibilityScene
import numpy as np
import torch
from data.PATHS import TMP_VIS_PATH
H = 375
W = 1242



pc = np.load("/home.dokt/vacekpa2/data/kittisf/all_data_format/000000.npz")['pc2']
pc = pc[:, [2, 0, 1]]
pc = torch.from_numpy(pc)
# pc[:, 1] = - pc[:, 1]
# pc[:, 2] = - pc[:, 2]

# visualize_points3D(pc, pc[:,2])

DepthScene = VisibilityScene(dataset='kitti_t',pc_scene=pc)


# todo from min
depth_image = DepthScene.depth_image

# flow
flow = pc.clone() #+ torch.rand(pc.shape) * 0.1 - 0.05

print(depth_image.shape)
visible_depth = DepthScene.assign_depth_to_flow(flow)

flow_depth = flow.norm(dim=1)

from vis.deprecated_vis import visualize_points3D
import matplotlib.pyplot as plt

visualize_points3D(flow, visible_depth > flow_depth)


plt.close()
plt.clf()
plt.cla()
fig, ax = plt.subplots(3,1)

ax[0].imshow(depth_image), ax[0].set_title('Orig depth')
plt.savefig(f'{TMP_VIS_PATH}/test_depth.png')


torch.Size([375, 1242])
